In [1]:
#scraping data from https://www.immobiliare.it/vendita-case/roma/?criterio=rilevanza&pag=1
import requests
from  bs4  import  BeautifulSoup
import pandas as pd
import time

In [2]:
#some urls are not completed. This fun add the missing part

def controlled(link):
    if link[:4] != 'http':
        link = 'https://www.immobiliare.it' + link 
    return link

In [4]:
#finding the announcements (more than 10000) 

links = []
idx = 1
count = 0
while (len(links)< 10000):

    url = 'https://www.immobiliare.it/vendita-case/roma/?criterio=rilevanza&pag=' + str(idx)
    idx += 1
    content = requests.get(url)
    soup = BeautifulSoup(content.text, "lxml")
    if count == 100:
        time.sleep(1)
        count = 0
        
    divTag = soup.find_all("div", {'class': "listing-item_body--content"})
    for tag in divTag:
        tdTags = tag.find_all("a")
        for tag in tdTags:
            l = controlled(tag['href'])
            links.append(l)
           

In [ ]:
#retrieving info for each url found before

r = pd.DataFrame(index = [0,1,2,3,4,5,6])
idx = 0
count = 1
for url in links: 
    try:
        content = requests.get(url)
        soup = BeautifulSoup(content.text, "lxml")
        row = []

        if count == 100:
            time.sleep(1)
            count = 0
        divTag = soup.find_all("h1", {'class': "raleway title-detail"})
        for title in divTag:
            row.append(title.text)

        divTag = soup.find_all("div", {'class': "clearfix description"})
        for desc in divTag:
            desc = desc.find_all("div")
            h = []
            for tag in desc:
                h.append((tag.text.replace('\n', ' ')))

        row.append(h[1])

        divTag = soup.find_all("div", {'class': "im-property__features"})
        for tag in divTag:

            price = tag.find_all("li", {'class': 'features__price'})
            for tag_p in price:
                row.append(((tag_p.text.replace('€', ' ')).split())[0])

            info = tag.find_all("ul", {'class': 'list-inline list-piped features__list'})
            h = []
            for tag in info:
                for tg in tag:
                    h.append((tg.text.replace('\xa0', ' ').replace('\n', ' ').replace('+', ' ')).split())


        for i in h[:-1]:
            if i[0] == 'da':
                row.append(i[1])
            else:
                row.append(i[0])
        r[idx] = pd.DataFrame(row)
        idx += 1
        
    except:
        print(idx)
        idx += 1

#save the dataframe in data.csv file         
r = r.transpose()
r.columns = ['title','description', 'price', 'locali', 'superficie', 'bagni', 'piano']
r.to_csv('data.csv', sep = ",",header= True, index = False)

In [16]:
f = pd.read_csv(r'data.csv')
f

,title,description,price,locali,superficie,bagni,piano
0,Nuovi Appartamenti e Attici / Mansarde a Roma,Le residenze...,192.000,1.0,46,NaN,NaN
1,"Bilocale viale Italo Calvino, Roma",PAPILLO EUR ...,225.000,2.0,50,1,1
2,"Quadrilocale via Mario Ridolfi 32, Roma",Rif: 18100 -...,359.000,4.0,130,3,3
3,"Appartamento via dei Gandolfi, Roma",In strada pr...,800.000,5.0,220,3,4
4,"Villa via Casalmonferrato, Roma",Nel quartier...,3.400.000,5.0,500,3,NaN
5,"Appartamento piazza Cavour, Roma","Prati, appar...",950.000,5.0,170,1,3
6,"Appartamento piazza Del Fante, Roma",In un contes...,1.090.000,5.0,190,2,2
7,"Attico / Mansarda via della Frezza, Roma",Nel contesto...,2.150.000,4.0,150,1,4
8,"Appartamento via Dandolo, Roma",Nella splend...,2.500.000,5.0,300,3,2
9,"Appartamento via del Calice, Roma",In via Del C...,650.000,5.0,193,3,4


In [25]:
f.groupby('piano').count()

,title,description,price,locali,superficie,bagni
piano,,,,,,
1,1249,1249,1249,1249,1249,1249
10,2,2,2,2,2,2
109,1,1,1,1,1,1
11,4,4,4,4,4,4
128,1,1,1,1,1,1
147,1,1,1,1,1,1
2,953,953,953,953,953,953
3,700,700,700,700,700,700
389,1,1,1,1,1,1
